# 🚀 Advanced Forecasting: Tuning, CV & Multi-Country
## Global Weather Repository - Phase 3 (Advanced)

---

**Objectives:**
1. **Hyperparameter Tuning**: Use Optuna for optimal XGBoost parameters
2. **Time Series Cross-Validation**: Walk-forward validation for robust evaluation
3. **Multi-Country Models**: Extend forecasting to ALL countries

---

## 1. Setup & Data Loading

In [1]:
# Core Libraries
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Visualization
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt

# Machine Learning
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.model_selection import TimeSeriesSplit
import xgboost as xgb

# Hyperparameter Tuning
import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances

# Prophet
from prophet import Prophet

# Utilities
import joblib
from tqdm import tqdm
import os

# Suppress Optuna logging
optuna.logging.set_verbosity(optuna.logging.WARNING)

print("✅ Libraries loaded successfully!")

✅ Libraries loaded successfully!


In [2]:
# Load processed data
DATA_PATH = '../data/processed/weather_with_anomalies.csv'
df = pd.read_csv(DATA_PATH)
df['date'] = pd.to_datetime(df['date'])
df['last_updated'] = pd.to_datetime(df['last_updated'])

print(f"📊 Dataset loaded: {len(df):,} rows")
print(f"🌍 Total Countries: {df['country'].nunique()}")

📊 Dataset loaded: 114,203 rows
🌍 Total Countries: 211


In [3]:
# Helper functions
def prepare_country_data(df, country):
    """Prepare daily time series for a specific country."""
    country_df = df[df['country'] == country].copy()
    
    daily_ts = country_df.groupby('date').agg({
        'temperature_celsius': 'mean',
        'humidity': 'mean',
        'pressure_mb': 'mean',
        'wind_kph': 'mean',
        'precip_mm': 'sum',
        'cloud': 'mean',
        'uv_index': 'mean',
        'is_anomaly': 'max'
    }).reset_index()
    
    daily_ts = daily_ts.sort_values('date').reset_index(drop=True)
    daily_ts.columns = ['date', 'temp', 'humidity', 'pressure', 'wind', 'precip', 'cloud', 'uv', 'is_anomaly']
    
    return daily_ts

def create_features(df, target_col='temp', lags=[1, 2, 3, 7]):
    """Create lag and rolling features for time series."""
    result = df.copy()
    
    for lag in lags:
        result[f'{target_col}_lag_{lag}'] = result[target_col].shift(lag)
    
    result[f'{target_col}_rolling_mean_7'] = result[target_col].rolling(window=7).mean()
    result[f'{target_col}_rolling_std_7'] = result[target_col].rolling(window=7).std()
    result[f'{target_col}_rolling_mean_14'] = result[target_col].rolling(window=14).mean()
    
    result['day_of_week'] = result['date'].dt.dayofweek
    result['day_sin'] = np.sin(2 * np.pi * result['day_of_week'] / 7)
    result['day_cos'] = np.cos(2 * np.pi * result['day_of_week'] / 7)
    
    return result.dropna().reset_index(drop=True)

def evaluate_model(y_true, y_pred):
    """Calculate evaluation metrics."""
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mape = mean_absolute_percentage_error(y_true, y_pred) * 100
    return {'mae': mae, 'rmse': rmse, 'mape': mape}

# Feature columns for XGBoost
FEATURE_COLS = [
    'humidity', 'pressure', 'wind', 'cloud', 'uv',
    'temp_lag_1', 'temp_lag_2', 'temp_lag_3', 'temp_lag_7',
    'temp_rolling_mean_7', 'temp_rolling_std_7', 'temp_rolling_mean_14',
    'day_sin', 'day_cos', 'is_anomaly'
]

print("✅ Helper functions defined")

✅ Helper functions defined


---
## 2. Time Series Cross-Validation

**Walk-Forward Validation**: Train on expanding window, test on next period.

```
Fold 1: [Train: ████░░░░░░] [Test: █░░░░░░░░░]
Fold 2: [Train: █████░░░░░] [Test: ░█░░░░░░░░]
Fold 3: [Train: ██████░░░░] [Test: ░░█░░░░░░░]
Fold 4: [Train: ███████░░░] [Test: ░░░█░░░░░░]
Fold 5: [Train: ████████░░] [Test: ░░░░█░░░░░]
```

In [4]:
# Prepare data for USA (for tuning)
TARGET_COUNTRY = 'United States of America'
daily_ts = prepare_country_data(df, TARGET_COUNTRY)
ts_data = create_features(daily_ts)

print(f"📊 {TARGET_COUNTRY}: {len(ts_data)} days of data")

📊 United States of America: 565 days of data


In [5]:
# Time Series Cross-Validation with sklearn
N_SPLITS = 5
tscv = TimeSeriesSplit(n_splits=N_SPLITS)

X = ts_data[FEATURE_COLS]
y = ts_data['temp']

# Visualize the splits
fig = go.Figure()

for fold, (train_idx, test_idx) in enumerate(tscv.split(X)):
    train_dates = ts_data.iloc[train_idx]['date']
    test_dates = ts_data.iloc[test_idx]['date']
    
    # Training period
    fig.add_trace(go.Scatter(
        x=[train_dates.min(), train_dates.max()],
        y=[fold + 1, fold + 1],
        mode='lines',
        line=dict(color='#4ECDC4', width=15),
        name=f'Fold {fold+1} Train' if fold == 0 else None,
        showlegend=(fold == 0)
    ))
    
    # Test period
    fig.add_trace(go.Scatter(
        x=[test_dates.min(), test_dates.max()],
        y=[fold + 1, fold + 1],
        mode='lines',
        line=dict(color='#FF6B6B', width=15),
        name=f'Fold {fold+1} Test' if fold == 0 else None,
        showlegend=(fold == 0)
    ))

fig.update_layout(
    title='📊 Time Series Cross-Validation Splits (Walk-Forward)',
    xaxis_title='Date',
    yaxis_title='Fold',
    height=350,
    yaxis=dict(tickmode='array', tickvals=list(range(1, N_SPLITS+1)))
)
fig.show()

In [6]:
# Perform Time Series CV with XGBoost (default params)
cv_results = []

print("🔄 Running Time Series Cross-Validation...")
print("="*60)

for fold, (train_idx, test_idx) in enumerate(tscv.split(X)):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    
    # Train XGBoost with default params
    model = xgb.XGBRegressor(
        n_estimators=100,
        max_depth=6,
        learning_rate=0.1,
        random_state=42,
        n_jobs=-1
    )
    model.fit(X_train, y_train, verbose=False)
    
    # Evaluate
    y_pred = model.predict(X_test)
    metrics = evaluate_model(y_test, y_pred)
    metrics['fold'] = fold + 1
    metrics['train_size'] = len(train_idx)
    metrics['test_size'] = len(test_idx)
    cv_results.append(metrics)
    
    print(f"Fold {fold+1}: MAE={metrics['mae']:.3f}°C, RMSE={metrics['rmse']:.3f}°C, MAPE={metrics['mape']:.2f}%")

cv_df = pd.DataFrame(cv_results)
print("\n" + "="*60)
print(f"📊 Mean MAE: {cv_df['mae'].mean():.3f} ± {cv_df['mae'].std():.3f}°C")
print(f"📊 Mean RMSE: {cv_df['rmse'].mean():.3f} ± {cv_df['rmse'].std():.3f}°C")
print(f"📊 Mean MAPE: {cv_df['mape'].mean():.2f} ± {cv_df['mape'].std():.2f}%")

🔄 Running Time Series Cross-Validation...
Fold 1: MAE=2.320°C, RMSE=3.128°C, MAPE=38.06%
Fold 2: MAE=2.611°C, RMSE=3.410°C, MAPE=249.13%
Fold 3: MAE=1.518°C, RMSE=1.997°C, MAPE=17.10%
Fold 4: MAE=1.669°C, RMSE=2.304°C, MAPE=11.82%
Fold 5: MAE=1.792°C, RMSE=2.239°C, MAPE=45.59%

📊 Mean MAE: 1.982 ± 0.464°C
📊 Mean RMSE: 2.616 ± 0.615°C
📊 Mean MAPE: 72.34 ± 99.83%


In [7]:
# Visualize CV results
fig = make_subplots(rows=1, cols=3, subplot_titles=['MAE per Fold', 'RMSE per Fold', 'MAPE per Fold'])

fig.add_trace(go.Bar(x=cv_df['fold'], y=cv_df['mae'], marker_color='#FF6B6B', name='MAE'), row=1, col=1)
fig.add_trace(go.Bar(x=cv_df['fold'], y=cv_df['rmse'], marker_color='#4ECDC4', name='RMSE'), row=1, col=2)
fig.add_trace(go.Bar(x=cv_df['fold'], y=cv_df['mape'], marker_color='#45B7D1', name='MAPE'), row=1, col=3)

fig.update_layout(title='📊 Cross-Validation Results by Fold', height=350, showlegend=False)
fig.show()

---
## 3. Hyperparameter Tuning with Optuna

Optuna uses Bayesian optimization to efficiently search the hyperparameter space.

In [8]:
def objective(trial):
    """
    Optuna objective function for XGBoost hyperparameter tuning.
    Uses Time Series CV for evaluation.
    """
    # Hyperparameter search space
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
        'random_state': 42,
        'n_jobs': -1
    }
    
    # Time Series CV
    tscv = TimeSeriesSplit(n_splits=3)  # Fewer splits for speed
    mae_scores = []
    
    for train_idx, test_idx in tscv.split(X):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        
        model = xgb.XGBRegressor(**params)
        model.fit(X_train, y_train, verbose=False)
        
        y_pred = model.predict(X_test)
        mae_scores.append(mean_absolute_error(y_test, y_pred))
    
    return np.mean(mae_scores)

In [9]:
# Run Optuna optimization
print("🔄 Running Optuna Hyperparameter Optimization...")
print("   This may take a few minutes...")

study = optuna.create_study(direction='minimize', study_name='xgboost_temp_forecast')
study.optimize(objective, n_trials=50, show_progress_bar=True)

print("\n" + "="*60)
print("✅ Optimization Complete!")
print(f"🏆 Best MAE: {study.best_value:.4f}°C")
print("\n📊 Best Hyperparameters:")
for key, value in study.best_params.items():
    print(f"   {key}: {value}")

🔄 Running Optuna Hyperparameter Optimization...
   This may take a few minutes...


Best trial: 42. Best value: 2.14006: 100%|██████████| 50/50 [00:06<00:00,  7.73it/s]


✅ Optimization Complete!
🏆 Best MAE: 2.1401°C

📊 Best Hyperparameters:
   n_estimators: 99
   max_depth: 3
   learning_rate: 0.10276229568344042
   subsample: 0.751092266382968
   colsample_bytree: 0.9692437478434137
   min_child_weight: 1
   reg_alpha: 0.0904099226880785
   reg_lambda: 5.783459007161972e-08


In [10]:
# Visualize optimization history
fig = go.Figure()

trials_df = study.trials_dataframe()

fig.add_trace(go.Scatter(
    x=trials_df['number'],
    y=trials_df['value'],
    mode='markers',
    marker=dict(color='#4ECDC4', size=8),
    name='Trial MAE'
))

# Best value line
best_values = [min(trials_df['value'][:i+1]) for i in range(len(trials_df))]
fig.add_trace(go.Scatter(
    x=trials_df['number'],
    y=best_values,
    mode='lines',
    line=dict(color='#FF6B6B', width=2),
    name='Best MAE'
))

fig.update_layout(
    title='📈 Optuna Optimization History',
    xaxis_title='Trial',
    yaxis_title='MAE (°C)',
    height=400
)
fig.show()

In [11]:
# Parameter importance
importance = optuna.importance.get_param_importances(study)

importance_df = pd.DataFrame([
    {'param': k, 'importance': v} for k, v in importance.items()
]).sort_values('importance', ascending=True)

fig = px.bar(
    importance_df, x='importance', y='param',
    orientation='h',
    title='🎯 Hyperparameter Importance',
    color='importance',
    color_continuous_scale='Viridis'
)
fig.update_layout(height=400)
fig.show()

In [12]:
# Compare Default vs Tuned XGBoost with full CV
print("📊 Comparing Default vs Tuned XGBoost...")
print("="*60)

# Default params
default_params = {
    'n_estimators': 100,
    'max_depth': 6,
    'learning_rate': 0.1,
    'random_state': 42,
    'n_jobs': -1
}

# Tuned params
tuned_params = {**study.best_params, 'random_state': 42, 'n_jobs': -1}

def run_cv(params, name):
    tscv = TimeSeriesSplit(n_splits=5)
    scores = []
    
    for train_idx, test_idx in tscv.split(X):
        model = xgb.XGBRegressor(**params)
        model.fit(X.iloc[train_idx], y.iloc[train_idx], verbose=False)
        y_pred = model.predict(X.iloc[test_idx])
        scores.append(mean_absolute_error(y.iloc[test_idx], y_pred))
    
    return {'model': name, 'mae_mean': np.mean(scores), 'mae_std': np.std(scores)}

default_result = run_cv(default_params, 'Default XGBoost')
tuned_result = run_cv(tuned_params, 'Tuned XGBoost')

comparison_df = pd.DataFrame([default_result, tuned_result])
print(comparison_df.to_string(index=False))

improvement = (default_result['mae_mean'] - tuned_result['mae_mean']) / default_result['mae_mean'] * 100
print(f"\n🚀 Improvement: {improvement:.2f}%")

📊 Comparing Default vs Tuned XGBoost...
          model  mae_mean  mae_std
Default XGBoost  1.982006 0.414582
  Tuned XGBoost  1.688964 0.220643

🚀 Improvement: 14.79%


---
## 4. Multi-Country Forecasting (ALL Countries)

Train models for **ALL countries** in the dataset that have sufficient data.

In [13]:
# Get ALL countries and their data availability
country_counts = df.groupby('country').size().sort_values(ascending=False)
ALL_COUNTRIES = country_counts.index.tolist()

print(f"🌍 Total Countries in Dataset: {len(ALL_COUNTRIES)}")
print(f"\n📊 Data Distribution:")
print(f"   Max records: {country_counts.max():,} ({country_counts.idxmax()})")
print(f"   Min records: {country_counts.min():,} ({country_counts.idxmin()})")
print(f"   Mean records: {country_counts.mean():,.0f}")
print(f"   Median records: {country_counts.median():,.0f}")

🌍 Total Countries in Dataset: 211

📊 Data Distribution:
   Max records: 1,327 (Bulgaria)
   Min records: 1 (Inde)
   Mean records: 541
   Median records: 586


In [14]:
# Visualize data distribution by country
fig = px.histogram(
    x=country_counts.values,
    nbins=30,
    title='📊 Distribution of Records per Country',
    labels={'x': 'Number of Records', 'y': 'Number of Countries'}
)
fig.update_traces(marker_color='#4ECDC4')
fig.update_layout(height=400)
fig.show()

In [15]:
# Train models for ALL countries
print("\n" + "="*70)
print(f"🔄 Training models for ALL {len(ALL_COUNTRIES)} countries...")
print("="*70)

# Minimum data requirements
MIN_DAILY_RECORDS = 20  # Minimum days of data needed
MIN_FEATURES_RECORDS = 15  # After feature engineering
MIN_TEST_RECORDS = 3  # Minimum test set size

multi_country_results = []
country_models = {}
skipped_countries = []

for country in tqdm(ALL_COUNTRIES, desc="Training countries"):
    try:
        # Prepare data
        daily_ts = prepare_country_data(df, country)
        
        if len(daily_ts) < MIN_DAILY_RECORDS:
            skipped_countries.append({'country': country, 'reason': 'Insufficient daily data', 'records': len(daily_ts)})
            continue
        
        ts_data = create_features(daily_ts)
        
        if len(ts_data) < MIN_FEATURES_RECORDS:
            skipped_countries.append({'country': country, 'reason': 'Insufficient data after features', 'records': len(ts_data)})
            continue
        
        # Train/test split (80/20)
        split_idx = int(len(ts_data) * 0.8)
        train = ts_data.iloc[:split_idx]
        test = ts_data.iloc[split_idx:]
        
        if len(test) < MIN_TEST_RECORDS:
            skipped_countries.append({'country': country, 'reason': 'Insufficient test data', 'records': len(test)})
            continue
        
        X_train = train[FEATURE_COLS]
        y_train = train['temp']
        X_test = test[FEATURE_COLS]
        y_test = test['temp']
        
        # Train with tuned params
        model = xgb.XGBRegressor(**tuned_params)
        model.fit(X_train, y_train, verbose=False)
        
        # Evaluate
        y_pred = model.predict(X_test)
        metrics = evaluate_model(y_test, y_pred)
        metrics['country'] = country
        metrics['train_size'] = len(train)
        metrics['test_size'] = len(test)
        metrics['total_records'] = country_counts[country]
        
        multi_country_results.append(metrics)
        country_models[country] = model
        
    except Exception as e:
        skipped_countries.append({'country': country, 'reason': str(e), 'records': 0})

# Summary
print("\n" + "="*70)
print(f"✅ Successfully trained: {len(country_models)} countries")
print(f"⚠️ Skipped: {len(skipped_countries)} countries (insufficient data)")


🔄 Training models for ALL 211 countries...


Training countries: 100%|██████████| 211/211 [00:07<00:00, 27.06it/s]


✅ Successfully trained: 186 countries
⚠️ Skipped: 25 countries (insufficient data)


In [16]:
# Create results dataframe
multi_results_df = pd.DataFrame(multi_country_results).sort_values('mae')

print("\n" + "="*70)
print("📊 MULTI-COUNTRY FORECASTING RESULTS (ALL COUNTRIES)")
print("="*70)
print(f"\nTotal models trained: {len(multi_results_df)}")
print(f"\n📊 Performance Statistics:")
print(f"   Best MAE:  {multi_results_df['mae'].min():.3f}°C ({multi_results_df.iloc[0]['country']})")
print(f"   Worst MAE: {multi_results_df['mae'].max():.3f}°C ({multi_results_df.iloc[-1]['country']})")
print(f"   Mean MAE:  {multi_results_df['mae'].mean():.3f}°C")
print(f"   Median MAE: {multi_results_df['mae'].median():.3f}°C")


📊 MULTI-COUNTRY FORECASTING RESULTS (ALL COUNTRIES)

Total models trained: 186

📊 Performance Statistics:
   Best MAE:  0.234°C (Maldives)
   Worst MAE: 3.039°C (Chad)
   Mean MAE:  1.386°C
   Median MAE: 1.256°C


In [17]:
# Show top 10 and bottom 10 countries
print("\n🏆 TOP 10 BEST PERFORMING COUNTRIES:")
print(multi_results_df[['country', 'mae', 'rmse', 'mape', 'train_size']].head(10).to_string(index=False))

print("\n⚠️ TOP 10 WORST PERFORMING COUNTRIES:")
print(multi_results_df[['country', 'mae', 'rmse', 'mape', 'train_size']].tail(10).to_string(index=False))


🏆 TOP 10 BEST PERFORMING COUNTRIES:
         country      mae     rmse     mape  train_size
        Maldives 0.234058 0.307048 0.858789         456
        Kiribati 0.265878 0.337315 0.964972         456
        Dominica 0.286283 0.407204 1.076001         456
         Somalia 0.355969 0.442326 1.250061         457
        Suriname 0.399962 0.520463 1.665948         456
Marshall Islands 0.424662 0.528289 1.526174         458
         Ecuador 0.443242 0.550447 4.245050         454
            Peru 0.444949 0.569398 2.629075         454
   Cote d'Ivoire 0.455312 0.586042 2.011827          90
     Timor-Leste 0.458369 0.593839 1.694141         456

⚠️ TOP 10 WORST PERFORMING COUNTRIES:
     country      mae     rmse         mape  train_size
 Switzerland 2.679417 3.567864 9.205190e+15         457
Vatican City 2.694309 3.531457 3.600010e+01         458
      Canada 2.785052 3.577659 1.511290e+02         456
     Namibia 2.820988 3.521578 1.197960e+01         457
   Australia 2.823792 3.8529

In [18]:
# Visualize ALL countries MAE distribution
fig = px.histogram(
    multi_results_df,
    x='mae',
    nbins=30,
    title=f'📊 MAE Distribution Across All {len(multi_results_df)} Countries',
    labels={'mae': 'MAE (°C)', 'count': 'Number of Countries'},
    color_discrete_sequence=['#4ECDC4']
)
fig.add_vline(x=multi_results_df['mae'].median(), line_dash='dash', line_color='red', annotation_text='Median')
fig.update_layout(height=400)
fig.show()

In [19]:
# Interactive bar chart - Top 30 best countries
top_30 = multi_results_df.head(30)

fig = px.bar(
    top_30,
    x='country',
    y='mae',
    color='mae',
    color_continuous_scale='Viridis',
    title='🏆 Top 30 Best Performing Countries (Lowest MAE)',
    labels={'mae': 'MAE (°C)', 'country': 'Country'}
)
fig.update_layout(xaxis_tickangle=-45, height=500)
fig.show()

In [20]:
# Scatter: MAE vs Training Size (ALL countries)
fig = px.scatter(
    multi_results_df,
    x='train_size',
    y='mae',
    color='mae',
    size='total_records',
    hover_name='country',
    color_continuous_scale='RdYlGn_r',
    title=f'📊 MAE vs Training Data Size (All {len(multi_results_df)} Countries)',
    labels={'train_size': 'Training Days', 'mae': 'MAE (°C)', 'total_records': 'Total Records'}
)
fig.update_layout(height=500)
fig.show()

In [21]:
# World map of MAE by country
fig = px.choropleth(
    multi_results_df,
    locations='country',
    locationmode='country names',
    color='mae',
    hover_name='country',
    hover_data=['mae', 'rmse', 'train_size'],
    color_continuous_scale='RdYlGn_r',
    title=f'🗺️ Forecasting Accuracy by Country (MAE °C) - {len(multi_results_df)} Countries'
)
fig.update_layout(height=600)
fig.show()

In [22]:
# Sample predictions for best and worst countries
best_country = multi_results_df.iloc[0]['country']
worst_country = multi_results_df.iloc[-1]['country']
median_idx = len(multi_results_df) // 2
median_country = multi_results_df.iloc[median_idx]['country']

fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=[
        f'🏆 Best: {best_country} (MAE: {multi_results_df.iloc[0]["mae"]:.2f}°C)',
        f'📊 Median: {median_country} (MAE: {multi_results_df.iloc[median_idx]["mae"]:.2f}°C)',
        f'⚠️ Worst: {worst_country} (MAE: {multi_results_df.iloc[-1]["mae"]:.2f}°C)'
    ]
)

for i, country in enumerate([best_country, median_country, worst_country]):
    daily_ts = prepare_country_data(df, country)
    ts_data = create_features(daily_ts)
    
    split_idx = int(len(ts_data) * 0.8)
    test = ts_data.iloc[split_idx:]
    
    X_test = test[FEATURE_COLS]
    y_pred = country_models[country].predict(X_test)
    
    fig.add_trace(
        go.Scatter(x=test['date'], y=test['temp'], mode='lines', name=f'Actual', line=dict(color='#2C3E50')),
        row=i+1, col=1
    )
    fig.add_trace(
        go.Scatter(x=test['date'], y=y_pred, mode='lines', name=f'Predicted', line=dict(color='#E74C3C', dash='dash')),
        row=i+1, col=1
    )

fig.update_layout(height=800, title='🎯 Best, Median, and Worst Country Predictions', showlegend=False)
fig.show()

In [23]:
# Show skipped countries
if skipped_countries:
    skipped_df = pd.DataFrame(skipped_countries)
    print(f"\n⚠️ Skipped Countries ({len(skipped_df)}):")
    print(skipped_df.groupby('reason').size().to_string())


⚠️ Skipped Countries (25):
reason
Insufficient daily data    25


---
## 5. Save All Artifacts

In [24]:
# Create directories
os.makedirs('../models', exist_ok=True)
os.makedirs('../models/countries', exist_ok=True)
os.makedirs('../reports', exist_ok=True)

# Save tuned parameters
tuned_params_df = pd.DataFrame([{'param': k, 'value': v} for k, v in study.best_params.items()])
tuned_params_df.to_csv('../models/xgboost_tuned_params.csv', index=False)
print("✅ Tuned parameters saved")

# Save multi-country results (ALL countries)
multi_results_df.to_csv('../reports/all_countries_results.csv', index=False)
print(f"✅ Results for {len(multi_results_df)} countries saved")

# Save skipped countries report
if skipped_countries:
    pd.DataFrame(skipped_countries).to_csv('../reports/skipped_countries.csv', index=False)
    print(f"✅ Skipped countries report saved ({len(skipped_countries)} countries)")

# Save ALL country models
print(f"\n🔄 Saving {len(country_models)} country models...")
for country, model in tqdm(country_models.items(), desc="Saving models"):
    safe_name = country.replace(' ', '_').replace('.', '').replace(',', '')
    joblib.dump(model, f'../models/countries/xgboost_{safe_name}.joblib')
print(f"✅ All {len(country_models)} country models saved to models/countries/")

# Save Optuna study
joblib.dump(study, '../models/optuna_study.joblib')
print("✅ Optuna study saved")

✅ Tuned parameters saved
✅ Results for 186 countries saved
✅ Skipped countries report saved (25 countries)

🔄 Saving 186 country models...


Saving models: 100%|██████████| 186/186 [00:00<00:00, 747.67it/s]

✅ All 186 country models saved to models/countries/
✅ Optuna study saved


---
## 📊 Summary

### Key Achievements:

1. **Time Series Cross-Validation**
   - Implemented walk-forward validation with 5 folds
   - Proper temporal ordering maintained (no future data leakage)

2. **Hyperparameter Tuning with Optuna**
   - Explored 50 trials across 8 hyperparameters
   - Found optimal configuration for XGBoost
   - Identified most important hyperparameters

3. **Multi-Country Forecasting (ALL Countries)**
   - Trained models for ALL available countries
   - Compared performance across different regions
   - Saved individual models for each country
   - Created global heatmap visualization

### Next Steps:
- Deploy models via FastAPI
- Add real-time data ingestion
- Implement model monitoring and retraining